In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip install simpletransformers

In [ ]:
project_path = '/content/drive/My Drive/Technical Debt/Codes/Stage2/Hierarchical/'## we will store our data in this drive

In [ ]:
def clean_text(text):
    
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)


    return text

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re

df_total=pd.read_excel(project_path+'TD_dataset.xlsx')
df_total = df_total[['Sentence Text','TD Type']]
df_total = df_total.rename({'Sentence Text': 'text', 'TD Type': 'labels'}, axis='columns')
df_total["labels"] = df_total["labels"].apply(lambda x: x.replace("REQUIREMENTS", "REQUIREMENT"))

df_total['text']=df_total['text'].map(lambda x: clean_text(x))





user = 'USER'
cran = 'CRAN'
developer = 'DEVELOPER'

type_parent_map = {'DOCUMENTATION':user, 'BUILD':cran, 'REQUIREMENT':user, 'ARCHITECTURE':cran, 'DESIGN':developer, 'USABILITY':user, 'CODE':developer, 'VERSIONING':cran, 'TEST':developer, 'DEFECT':developer}


type_label_map = {'DOCUMENTATION':0, 'BUILD':-1, 'REQUIREMENT':-1, 'ARCHITECTURE':-1, 'DESIGN':1, 'USABILITY':-1, 'CODE':2, 'VERSIONING':-1, 'TEST':3, 'DEFECT':4}



parent_label_map = {user:0, developer:1, cran:2}





#print(type_label_map['BUILD'])

df_total_label_list = df_total['labels'].tolist()

df_total_label_numeric_list = []

for label_type in df_total_label_list:
  numeric_label = type_label_map[label_type]
  df_total_label_numeric_list.append(numeric_label)

df_total['labels'] = df_total_label_numeric_list

del df_total_label_list
del df_total_label_numeric_list


df_total = df_total[df_total['labels']!=-1]

num_of_TD_types = len(set(df_total['labels'].tolist()))

df_train, df_test = train_test_split(df_total, test_size=0.2, random_state=1)



texts_train=[]
texts_train=df_train['text']

label_train=df_train['labels']


X_train=texts_train.astype(str).values
X_train=np.reshape(X_train,(-1,1))


label_train=label_train.astype(int).values
y_train=np.reshape(label_train,(-1,1))


In [ ]:


texts_test=[]
texts_test=df_test['text']
label_test=df_test['labels']


X_test=texts_test.astype(str).values
X_test=np.reshape(X_test,(-1,1))


label_test=label_test.astype(int).values
y_test=np.reshape(label_test,(-1,1))

In [ ]:
sentences = X_train[:,0]
print(sentences[0])
labels = y_train[:, 0]
#labels = y_train
print(labels[0])

In [ ]:
VALIDATION_SPLIT = 0.2

nb_validation_samples = int(VALIDATION_SPLIT * sentences.shape[0])

x_train = sentences[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = sentences[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print(x_train[0])
print(x_val[0])

In [ ]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
sentences_test = X_test[:,0]
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test[0])

In [ ]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
model_type = "bert"
model_name = "bert-base-cased"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": project_path+"output2/"+model_type,
    "best_model_dir": project_path+"output2/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News Liar_1 Bert",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["evaluate_during_training_steps"] = 256
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = True


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name,num_labels=num_of_TD_types, args=train_args)

# Train the model
model.train_model(train_df,eval_df=test_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [ ]:
predictions, raw_outputs = model.predict(test_df['text'])


In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test_df['labels'].tolist(),predictions,labels=[0,1,2,3,4,5,6,7,8,9]))

#'DOCUMENTATION':0, 'BUILD':1, 'REQUIREMENT':2, 'ARCHITECTURE':3, 'DESIGN':4, 'USABILITY':5, 'CODE':6, 'VERSIONING':7, 'TEST':8, 'DEFECT':9

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y_true, y_pred, average='weighted'))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


loaded_model = ClassificationModel(
    "bert", project_path + "output2/bert/"
)

predictions_by_loaded_model, raw_outputs_loaded_model = model.predict(test_df['text'])

from sklearn.metrics import accuracy_score


print(accuracy_score(test_df['labels'],predictions_by_loaded_model))
